## connect to weaviate

In [1]:
import weaviate
client = weaviate.connect_to_local()

/Users/jbc/Documents/SevenSix/etl_data_processing/.venv/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/aggregate.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/jbc/Documents/SevenSix/etl_data_processing/.venv/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/base.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/jbc/Documents/SevenSix/etl_data_processing/.venv/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/base_search.proto. Please update 

## Check existing collections

In [2]:
## get all collections
client.collections.list_all()

{'DemoCollection': _CollectionConfigSimple(name='DemoCollection', description=None, generative_config=None, properties=[], references=[], reranker_config=None, vectorizer_config=None, vectorizer=None, vector_config={'title_vector': _NamedVectorConfig(vectorizer=_NamedVectorizerConfig(vectorizer=<Vectorizers.TEXT2VEC_TRANSFORMERS: 'text2vec-transformers'>, model={'poolingStrategy': 'masked_mean', 'vectorizeClassName': True}, source_properties=['title']), vector_index_config=_VectorIndexConfigHNSW(multi_vector=None, quantizer=None, cleanup_interval_seconds=300, distance_metric=<VectorDistances.COSINE: 'cosine'>, dynamic_ef_min=100, dynamic_ef_max=500, dynamic_ef_factor=8, ef=-1, ef_construction=128, filter_strategy=<VectorFilterStrategy.SWEEPING: 'sweeping'>, flat_search_cutoff=40000, max_connections=32, skip=False, vector_cache_max_objects=1000000000000))}),
 'Docling': _CollectionConfigSimple(name='Docling', description=None, generative_config=None, properties=[], references=[], rerank

In [ ]:
## connect single collection
collection = client.collections.get("DemoCollection")
print("config of collection:",collection.config.get())
print("properties :",collection.config.get().properties)

config of collection: _CollectionConfig(name='DemoCollection', description=None, generative_config=None, inverted_index_config=_InvertedIndexConfig(bm25=_BM25Config(b=0.75, k1=1.2), cleanup_interval_seconds=60, index_null_state=False, index_property_length=False, index_timestamps=False, stopwords=_StopwordsConfig(preset=<StopwordsPreset.EN: 'en'>, additions=None, removals=None)), multi_tenancy_config=_MultiTenancyConfig(enabled=False, auto_tenant_creation=False, auto_tenant_activation=False), properties=[], references=[], replication_config=_ReplicationConfig(factor=1, async_enabled=False, deletion_strategy=<ReplicationDeletionStrategy.NO_AUTOMATED_RESOLUTION: 'NoAutomatedResolution'>), reranker_config=None, sharding_config=_ShardingConfig(virtual_per_physical=128, desired_count=1, actual_count=1, desired_virtual_count=128, actual_virtual_count=128, key='_id', strategy='hash', function='murmur3'), vector_index_config=None, vector_index_type=None, vectorizer_config=None, vectorizer=None

## Create connection
Weaviate follows GraphQL naming conventions. 
- Start collection names with an upper case letter.
- Start property names with a lower case letter.

N.T: if mismatch GraphQL format, weaviate chates it internally.

### Production ready collections
1. **Manually define you data schema:-** Avoid using the auto-schema feature, instead, manually define the properties for your collection.
2. **Avoid creating too many collections:-** Using too many collections can lead to scalability issues like high memory usage and degraded query performance. Instead, consider using multi-tenancy, where a single collection is subdivided into multiple tenants.

In [ ]:
import weaviate.classes.config as wc
collection  = client.collections.create(
    name="DemoCollection1",
    properties=[
            wc.Property(name="name", data_type=wc.DataType.TEXT),
            wc.Property(name="age", data_type=wc.DataType.INT),
            wc.Property(name="email", data_type=wc.DataType.TEXT),
        ],
    vector_config=wc.Configure.Vectors.text2vec_transformers(
        name="text_vector",
        source_properties=["text"]
    )
    )
collection.config.get()

_CollectionConfig(name='DemoCollection1', description=None, generative_config=None, inverted_index_config=_InvertedIndexConfig(bm25=_BM25Config(b=0.75, k1=1.2), cleanup_interval_seconds=60, index_null_state=False, index_property_length=False, index_timestamps=False, stopwords=_StopwordsConfig(preset=<StopwordsPreset.EN: 'en'>, additions=None, removals=None)), multi_tenancy_config=_MultiTenancyConfig(enabled=False, auto_tenant_creation=False, auto_tenant_activation=False), properties=[_Property(name='name', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer=None, vectorizer_configs={'text2vec-transformers': _PropertyVectorizerConfig(skip=False, vectorize_property_name=False)}), _Property(name='age', description=None, data_type=<DataType.INT: 'int'>, index_filterable=True, index_range_filters=False, index_searchabl

In [23]:
### Add a new property to the collection
import weaviate.classes.config as wc
collection.config.add_property(
    wc.Property(
        name="new_",
        data_type=wc.DataType.TEXT,
        description="A new property added to the collection"
    )
)
print("properties :",collection.config.get().properties)

properties : [_Property(name='name', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer=None, vectorizer_configs={'text2vec-transformers': _PropertyVectorizerConfig(skip=False, vectorize_property_name=False)}), _Property(name='age', description=None, data_type=<DataType.INT: 'int'>, index_filterable=True, index_range_filters=False, index_searchable=False, nested_properties=None, tokenization=None, vectorizer_config=None, vectorizer=None, vectorizer_configs={'text2vec-transformers': _PropertyVectorizerConfig(skip=False, vectorize_property_name=False)}), _Property(name='email', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer=None, vectoriz

In [ ]:
#### update the collection definition

import weaviate.classes.config as wc

articles = client.collections.get("DemoCollection1")

# Update the collection definition
articles.config.update(
    description="An updated collection description.",
    # property_descriptions={
    #     "title": "The updated title description for article",
    # },  # Available from Weaviate v1.31.0
    inverted_index_config=wc.Reconfigure.inverted_index(bm25_b=0.75,  
        bm25_k1=1.5,  
    ),
 
    vector_config=wc.Reconfigure.Vectors.update(
        name="text_vector",
        vector_index_config=wc.Reconfigure.VectorIndex.hnsw(
            filter_strategy=wc.VectorFilterStrategy.ACORN  # Available from Weaviate v1.27.0
        ),
    ),
    replication_config=wc.Reconfigure.replication(
        deletion_strategy=wc.ReplicationDeletionStrategy.TIME_BASED_RESOLUTION  # Available from Weaviate v1.28.0
    ),
)


In [ ]:
articles = client.collections.get("DemoCollection1")

article_shards = articles.config.update_shards(
    status="READY",
    # shard_names=["shard_names"],  # The names (List[str]) of the shard to update (or a shard name)
)
print(article_shards)

{'36h64iLx7vL9': 'READY'}


## Delete Collections

In [34]:
## single collection delete
client.collections.delete("DemoCollection1")
## delete all collections
client.collections.delete_all()

In [35]:
client.collections.list_all()

{}